In [ ]:
from tkinter import filedialog
from tkinter import messagebox
import tkinter as tk
import csv
import re
import unicodedata
import os


In [ ]:
#https://stackoverflow.com/a/42398049


WINDOW_TITLE_KEY="Selecciona el fichero CSV en inglés"
WINDOW_TITLE="Selecciona todos los ficheros CSV"
FILE_TYPE_TITLE="Fichero CSV"
FILE_TYPE=".csv"

root = tk.Tk()
# Hide the main window
root.withdraw()
root.call('wm', 'attributes', '.', '-topmost', True)

events_key_file=filedialog.askopenfilename(filetypes=[(FILE_TYPE_TITLE,FILE_TYPE)],title=WINDOW_TITLE_KEY)
files=filedialog.askopenfilenames(filetypes=[(FILE_TYPE_TITLE,FILE_TYPE)],title=WINDOW_TITLE)
%gui tk

In [ ]:

'''Convierte un string en mayusculas, los espacios se cambian por _ y solo se aceptan letras del abecedario y numeros.
Por ejemplo Submission updated. se parsea como SUBMISSION_UPDATED
'''
def parse(string):
    #Cambiamos las letras con acento por uno sin acento, fuente: https://stackoverflow.com/a/15261831
    parsed=''.join((c for c in unicodedata.normalize('NFD', string) if unicodedata.category(c) != 'Mn'))
    parsed=re.sub(" ","_",parsed) #cambiamos los espacios por _
    parsed=re.sub("[^\w]+","",parsed) #eliminamos todos lo que no sea letra alfabetica ni numero
    return parsed.upper() #to upper

In [ ]:
EVENT_NAME="Event name"
COMPONENT="Component"
EVENT_KEY="Event key"
EVENT_PATH="Event path"
COMPONENT_KEY="Component key"
EDUCATION_LEVEL="Education level"
DATABASE_QUERY_TYPE="Database query type"
AFFECTED_TABLE="Affected table"
SINCE="Since"
LEGACY_EVENT="Legacy event"
headers=[EVENT_KEY,COMPONENT_KEY,EVENT_PATH,EDUCATION_LEVEL,DATABASE_QUERY_TYPE,AFFECTED_TABLE,SINCE,LEGACY_EVENT]

In [ ]:
with open(events_key_file,mode="r",encoding="utf-8") as file:
    reader=csv.DictReader(file, delimiter=',')
    lista = []
    for r in reader:
        row={}
        event, path=r[EVENT_NAME].split("\n",1)
        row[EVENT_KEY] = parse(event)
        row[EVENT_PATH]=path
        row[COMPONENT_KEY] = parse(r[COMPONENT] if r[COMPONENT]!="core" else "System")
        row[EDUCATION_LEVEL] = r[EDUCATION_LEVEL]
        row[DATABASE_QUERY_TYPE]= r[DATABASE_QUERY_TYPE]
        row[AFFECTED_TABLE] =r[AFFECTED_TABLE]
        row[SINCE]= r[SINCE]
        row[LEGACY_EVENT]= r[LEGACY_EVENT]
        lista.append(row)
        

In [ ]:
file_names=[]
for path_file in files:
    with open(path_file, mode="r", encoding="utf-8") as file:
        base=os.path.basename(path_file)
        file_name=os.path.splitext(base)[0]
        file_names.append(COMPONENT+"_"+file_name)
        file_names.append(EVENT_NAME+"_"+file_name)
        reader=csv.reader(file, delimiter=',')
        next(reader) #ignoramos el header
        for l, row in zip(lista,reader):
            l[EVENT_NAME+"_"+file_name]=row[0].split("\n",1)[0]
            l[COMPONENT+"_"+file_name]=row[1] if row[1]!="core" else "System"

In [ ]:
with open("lista de eventos.csv",mode="w",encoding="utf-8") as csv_file:
    fieldnames =headers +file_names
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames,lineterminator='\n')
    writer.writeheader()
    for row in lista:
        writer.writerow(row)
    